# Get Point-in-time datasets
Find all of the dates when companies first published their quarterly reports

In [87]:
import bql
import pandas as pd
import helper
import boto3
import json
from s3fs import S3FileSystem
import os

from ipywidgets import IntProgress
from IPython.display import display

In [18]:
import importlib
importlib.reload(helper)

<module 'helper' from '/project/helper.py'>

In [3]:
bq = bql.Service()

In [4]:
# Index to use for point in time firms
index = 'SPX Index'

# rebalance dates for the index
rebalance_dates = ['2023-12-31',
        '2023-09-30',
        '2023-06-30',
        '2023-03-31',
        '2022-12-31',
        '2022-09-30',
        '2022-06-30',
        '2022-03-31',
        '2021-12-31',
        '2021-09-30',
        '2021-06-30',
        '2021-03-31',
        '2020-12-31',
        '2020-09-30',
        '2020-06-30',
        '2020-03-31',
        '2019-12-31',
        '2019-09-30',
        '2019-06-30',
        '2019-03-31',]


# Calculate the Rebalance dates

For each rebalance date, get the members of the index at that time and look up their reporting dates

In [5]:
all_data = []

def get_reporting_dates_per_rebalance(date):
    univ = bq.univ.members(index, dates=date)
    field = bq.data.sales_rev_turn(dates=bq.func.range('-5Y','0D'), fa_period_type='Q')
    req = bql.Request(univ, field)
    data = bq.execute(req)
    df = data[0].df().dropna()
    return df.sort_values('PERIOD_END_DATE', ascending=True).reset_index().drop_duplicates(subset=['ID','PERIOD_END_DATE'], keep='first')

In [6]:
def get_rebalance_dates():
    for date in rebalance_dates:
        all_data.append(get_reporting_dates_per_rebalance(date))
        print("Complete for ", date)
    df = pd.concat(all_data)
    df_concat = df[['ID','AS_OF_DATE','PERIOD_END_DATE']].sort_values('PERIOD_END_DATE', ascending=True).drop_duplicates(subset=['ID','PERIOD_END_DATE'], keep='first')
    return df_concat.set_index(['AS_OF_DATE','ID']).sort_values(['AS_OF_DATE'])

In [7]:
df_rebalance_dates = get_rebalance_dates()

Complete for  2023-12-31
Complete for  2023-09-30
Complete for  2023-06-30
Complete for  2023-03-31
Complete for  2022-12-31
Complete for  2022-09-30
Complete for  2022-06-30
Complete for  2022-03-31
Complete for  2021-12-31
Complete for  2021-09-30
Complete for  2021-06-30
Complete for  2021-03-31
Complete for  2020-12-31
Complete for  2020-09-30
Complete for  2020-06-30
Complete for  2020-03-31
Complete for  2019-12-31
Complete for  2019-09-30
Complete for  2019-06-30
Complete for  2019-03-31


In [81]:
# Convert datasets to dictionary
def convert_to_dict(securities, df_is, df_bs, df_px):
    date = {}
    data = {}
    for security in securities:
        data['is'] = df_is.loc[security].to_string()
        data['bs'] = df_bs.loc[security].to_string()
        data['px'] = df_px.loc[security].set_index('DATE')[['Price']].to_string()
        date[security] = data
    return date


def process_single_date(securities, fields):
    req = bql.Request(securities, fields)
    data = bq.execute(req)
    if len(fields) > 1:
        return format_request_to_df(data, fields)
    else:
        return data[0].df()


# main request for financial data
def update_financial_data(dates_and_securities):
    all_data = {}
    is_first = True
    dates = dates_and_securities.reset_index()['AS_OF_DATE'].unique()
    max_count = len(dates)
    f = IntProgress(min=0, max=max_count) # instantiate the bar
    display(f)
    # Loop through each date and extract securities
    for date in dates:
        if is_first:
            is_first=False
        else:
            as_of_date = str(date)[0:10]
            securities = list(dates_and_securities.loc[as_of_date].index)
            univ, is_fields, bs_fields, price = helper.setup_request(securities, as_of_date) 
            try:
                df_is = process_single_date(securities, is_fields)
                df_bs = process_single_date(securities, bs_fields)
                df_px = process_single_date(securities, price)
                all_data[as_of_date] = convert_to_dict(securities, df_is, df_bs, df_px)
            except:
                print(as_of_date)
            f.value += 1
    return all_data
        

In [82]:
all_data = update_financial_data(df_rebalance_dates)

IntProgress(value=0, max=1047)

2020-08-17
2020-09-21
2021-08-16
2021-09-24
2022-09-28
2022-11-25
2023-10-25
2024-01-08
2024-06-17
2024-11-08
2025-01-08


In [88]:
## Save to S3
user_bucket_name = os.environ['BQUANT_SANDBOX_USER_BUCKET']
bqnt_username = os.environ['BQUANT_USERNAME']

s3 = boto3.resource("s3")

path_to_s3 = f's3://{user_bucket_name}/{bqnt_username}/tmp/fs/data_quarterly_pit.json'
s3 = S3FileSystem()

with s3.open(path_to_s3, 'w') as file:
    json.dump(all_data, file)

In [8]:
# pull out the list of as of dates - will use this to loop through the securities
dates = df_rebalance_dates.reset_index()['AS_OF_DATE'].unique()

### Get each as of date and request the securities

In [71]:
as_of_date = '2020-03-11'#str(dates[12])[0:10]
securities = list(df_rebalance_dates.loc[as_of_date].index)
as_of_date

'2020-03-11'

In [72]:
len(securities)

2

In [52]:
len(dates)

1047

In [73]:
univ, is_fields, bs_fields, price = helper.setup_request(securities, as_of_date)

In [74]:
req = bql.Request(securities,is_fields)
data = bq.execute(req)

In [80]:
# Convert the request into correct format for data frame
def format_request_to_df(data, fields):
    fields = list(fields.keys())
    df_all = [data[index].df()[data[index].df()['PERIOD_END_DATE'] != 0]
                  .pivot(columns='PERIOD_END_DATE', values=[fields[index]])
                  .fillna(0) 
                  for index in range(0,len(fields))]
    df2 = pd.concat(df_all, axis=1)
    df3 = df2.stack().transpose().stack().unstack(level=0).transpose().fillna(0)
    df4 = df3.loc[:, (df3 != 0).any(axis=0)]
    # Reformat the columns to remove dates
    if len(df4.columns) == 6:
        df5 = df4.set_axis(['t-5','t-4','t-3','t-2','t-1', 't'], axis='columns')
    else:
        df5 = df4.drop(columns=df4.columns[0:(len(df4.columns)-6)])
        df5 = df5.set_axis(['t-5','t-4','t-3','t-2','t-1', 't'], axis='columns')
    # Reverse the direction of the dataset
    df6 = df5[df5.columns[::-1]]
    return df6.loc[(df6!=0).any(axis=1)]
        

In [60]:
# test run
req = bql.Request(securities, is_fields)
data_is = bq.execute(req)
req = bql.Request(securities, bs_fields)
data_bs = bq.execute(req)
df_is = format_request_to_df(data_is, is_fields)
df_bs = format_request_to_df(data_bs, bs_fields)

ValueError: Length mismatch: Expected axis has 6 elements, new values have 7 elements

In [75]:
fields = list(is_fields.keys())
df_all = [data[index].df()[data[index].df()['PERIOD_END_DATE'] != 0]
              .pivot(columns='PERIOD_END_DATE', values=[fields[index]])
              .fillna(0) 
              for index in range(0,len(fields))]
df2 = pd.concat(df_all, axis=1)
df3 = df2.stack().transpose().stack().unstack(level=0).transpose().fillna(0)
df4 = df3.loc[:, (df3 != 0).any(axis=0)]

In [77]:
df4.columns

DatetimeIndex(['2018-09-30', '2018-12-31', '2019-03-31', '2019-06-30',
               '2019-09-30', '2019-12-31', '2020-03-31'],
              dtype='datetime64[ns]', name='PERIOD_END_DATE', freq=None)

In [79]:
df4.drop(columns=df4.columns[0:(len(df4.columns)-6)])#drop(df4.columns[0],axis=1)

PERIOD_END_DATE                                                  2018-12-31  \
ID                                                                            
COST UW Equity 01 Revenue (Adj)                                3.506900e+10   
               02 Sales and Services Revenues (Adj)            3.431100e+10   
               03 Financing Revenue (Adj)                      0.000000e+00   
               04 Other Revenue (Adj)                          7.580000e+08   
               05 Cost of Revenue (Adj)                        3.062300e+10   
...                                                                     ...   
TGT UN Equity  47 Basic EPS from Continuing Operations         1.540681e+00   
               48 Diluted Weighted Average Shares              0.000000e+00   
               49 Diluted EPS                                  1.520000e+00   
               50 Diluted EPS from Continuing Operations       1.520000e+00   
               51 Diluted EPS from Continuing Operations, Adj  1.525722e+00   

PERIOD_END_DATE                                                  2019-03-31  \
ID                                                                            
COST UW Equity 01 Revenue (Adj)                                3.539600e+10   
               02 Sales and Services Revenues (Adj)            3.462800e+10   
               03 Financing Revenue (Adj)                      0.000000e+00   
               04 Other Revenue (Adj)                          7.680000e+08   
               05 Cost of Revenue (Adj)                        3.072000e+10   
...                                                                     ...   
TGT UN Equity  47 Basic EPS from Continuing Operations         1.540000e+00   
               48 Diluted Weighted Average Shares              0.000000e+00   
               49 Diluted EPS                                  1.530000e+00   
               50 Diluted EPS from Continuing Operations       1.530000e+00   
               51 Diluted EPS from Continuing Operations, Adj  1.530000e+00   

PERIOD_END_DATE                                                  2019-06-30  \
ID                                                                            
COST UW Equity 01 Revenue (Adj)                                3.474000e+10   
               02 Sales and Services Revenues (Adj)            3.396400e+10   
               03 Financing Revenue (Adj)                      0.000000e+00   
               04 Other Revenue (Adj)                          7.760000e+08   
               05 Cost of Revenue (Adj)                        3.023300e+10   
...                                                                     ...   
TGT UN Equity  47 Basic EPS from Continuing Operations         1.830000e+00   
               48 Diluted Weighted Average Shares              0.000000e+00   
               49 Diluted EPS                                  1.820000e+00   
               50 Diluted EPS from Continuing Operations       1.820000e+00   
               51 Diluted EPS from Continuing Operations, Adj  1.820000e+00   

PERIOD_END_DATE                                                  2019-09-30  \
ID                                                                            
COST UW Equity 01 Revenue (Adj)                                4.749800e+10   
               02 Sales and Services Revenues (Adj)            4.644800e+10   
               03 Financing Revenue (Adj)                      0.000000e+00   
               04 Other Revenue (Adj)                          1.050000e+09   
               05 Cost of Revenue (Adj)                        4.131000e+10   
...                                                                     ...   
TGT UN Equity  47 Basic EPS from Continuing Operations         1.384206e+00   
               48 Diluted Weighted Average Shares              0.000000e+00   
               49 Diluted EPS                                  1.390000e+00   
               50 Diluted EPS from Continuing O

#### 

In [21]:
df_is.loc[securities[0]]

,t,t-1,t-2,t-3,t-4,t-5
01 Revenue (Adj),4.150900e+09,4.344300e+09,4.527800e+09,3.575900e+09,3.895100e+09,4.030900e+09
02 Sales and Services Revenues (Adj),4.150900e+09,4.344300e+09,4.527800e+09,3.575900e+09,3.895100e+09,4.030900e+09
05 Cost of Revenue (Adj),2.904300e+09,2.935800e+09,3.094100e+09,2.517300e+09,2.745000e+09,2.718300e+09
06 Cost of Goods & Services Sold (Adj),2.904300e+09,2.935800e+09,3.094100e+09,2.517300e+09,2.745000e+09,2.718300e+09
08 Gross Profit (Adj),1.246600e+09,1.408500e+09,1.433700e+09,1.058600e+09,1.150100e+09,1.312600e+09
10 Operating Expenses (Adj),8.212000e+08,7.853000e+08,7.832000e+08,7.401000e+08,7.034000e+08,7.256000e+08
"11 Selling, General and Administrative Expense (Adj)",8.212000e+08,7.853000e+08,7.832000e+08,7.401000e+08,7.034000e+08,7.256000e+08
14 Operating Income or Losses (Adj),4.254000e+08,6.232000e+08,6.505000e+08,3.185000e+08,4.467000e+08,5.870000e+08
15 Non-Operating (Income) Loss (Adj),7.370000e+07,7.130000e+07,6.130000e+07,6.970000e+07,6.940000e+07,5.700000e+07
16 Net Interest Expense (Adj),0.000000e+00,6.370000e+07,6.150000e+07,5.030000e+07,4.890000e+07,4.840000e+07


In [22]:
t = convert_to_dict(securities, df_is, df_bs, df_is)

In [24]:
t[securities[0]]['is']

'                                                                 t           t-1           t-2           t-3           t-4           t-5\n01 Revenue (Adj)                                      6.497870e+08  6.073410e+08  6.006970e+08  5.489710e+08  5.340200e+08  5.052890e+08\n02 Sales and Services Revenues (Adj)                  6.497870e+08  6.073410e+08  6.006970e+08  5.489710e+08  5.340200e+08  5.052890e+08\n05 Cost of Revenue (Adj)                              1.778290e+08  1.697870e+08  1.684080e+08  1.468750e+08  1.509240e+08  1.335080e+08\n06 Cost of Goods & Services Sold (Adj)                1.778290e+08  1.697870e+08  1.684080e+08  1.468750e+08  1.509240e+08  1.335080e+08\n08 Gross Profit (Adj)                                 4.719580e+08  4.375540e+08  4.322890e+08  4.020960e+08  3.830960e+08  3.717810e+08\n10 Operating Expenses (Adj)                           3.208080e+08  3.104020e+08  2.557990e+08  3.143950e+08  2.626230e+08  2.465730e+08\n11 Selling, General and Administr

In [25]:
univ = securities
field = {'price': bq.data.px_last(dates=bq.func.range('-12M', as_of_date), currency='USD', fill='prev')}

req = bql.Request(univ, field)
data = bq.execute(req)

In [32]:
data[0].df().loc['TT UN Equity'].set_index('DATE')[['price']]

,price
DATE,
2019-01-29,75.056425
2019-02-28,81.891020
2019-03-29,83.745127
2019-04-29,93.450097
2019-05-29,91.751146
2019-06-29,98.267673
2019-07-29,94.815465
2019-08-29,93.232880
2019-09-29,95.855006
